In [191]:
import numpy as np
import tensorflow as tf
import os
import time
import nltk
from collections import Counter

Defining the directories for training and test sets

In [192]:
train_path = "./aclImdb/train/"
test_path = "./aclImdb/test/"

Loading all filenames for training set

In [193]:
train_filenames = [train_path+"pos/"+i for i in (os.listdir(train_path+"pos"))] + [train_path+"neg/"+i for i in (os.listdir(train_path+"neg"))]

In [194]:
prop_noun = "<pn>"
end_sentence = "<end_sent>"
separator = "<sep>"

Helper function who read stopwords from file

In [195]:
def stopwords(file_path):
    ret = set()
    file = open(file_path,encoding="utf8")
    for line in file:
        ret.add(line[0:-1])
    return ret

Some types of words

In [196]:
end_sentence_tokens = [".","?","!"]
separator_tokens=[",",";"]
another_stopwords = stopwords("./aclImdb/train/stopwords.txt")

In [197]:
dictionary = {}
counter = Counter()
count = 0
max_sequence_lenght = 0

Creating a dictionary of the whole training set

In [198]:
for file in train_filenames:
    content = open(file,encoding="utf8").read()
    words = nltk.tokenize.word_tokenize(content)
    max_sequence_lenght=max(max_sequence_lenght,len(words))
    for word in words:
        if not (word in dictionary):
            dictionary[word]=count
            count+=1
        counter.update([word]);

            

In [199]:
copy_counter = Counter(counter)

In [200]:
len(dictionary)

134448

In [201]:
max_sequence_lenght

2818

Helper function who classify word according to part of speech tag

In [203]:
def classify_word(word_type):
    wt = word_type
    if wt[0:2]=="VB" or wt[0:2]=="MD":
        return 'v'
    if wt[0:2]=="NN" or wt[0:2]=="PO":
        return "n"
    if wt[0:2]=="JJ":
        return "a"
    if wt[0:2]=="WP" or wt[0:2]=="WD" or wt[0:2]=="PR" or wt[0:2]=="CD" or wt[0:2]=="IN" or wt[0:2]=="FW" or wt[0:2]=="PD":
        return "s"
    if wt[0:2]=="RB":
        return "a"
    return ""

In [204]:
l=[*dictionary]
tagged = nltk.pos_tag(l)

In [205]:
counter=Counter(copy_counter)

In [206]:
from nltk.stem import WordNetLemmatizer
 
wnl = WordNetLemmatizer()

types = {}
new_dict = {}
reversed_dict = {}
count=1
for word in tagged:
    types[word[0]] = classify_word(word[1])
    if types[word[0]]!="s" and not word[0].lower() in another_stopwords:
        if types[word[0]]=="":
            w = wnl.lemmatize(word[0])
        else:
            w = wnl.lemmatize(word[0], types[word[0]])
        if w in end_sentence_tokens:
            w=end_sentence
        if w in separator_tokens:
            w=separator
        if word[1][0:3]=="NNP":
            w=prop_noun
        w = w.lower()
        if w=="n't":
            w="not"
        if word[0]!=w:
            counter.update({w:counter.pop(word[0])})
        new_dict[w] = count
        reversed_dict[count] = w
        count+=1
        dictionary[word[0]]=new_dict[w]
            
            
    else:
        counter.pop(word[0])
        dictionary[word[0]]=-1

In [207]:
len(counter)

67351

In [208]:
counter.most_common(1000)

[('<pn>', 547331),
 ('<sep>', 282589),
 ('<end_sent>', 277820),
 ('not', 63976),
 ('movie', 49129),
 ('film', 45975),
 ('make', 22285),
 ('good', 20175),
 ('so', 20021),
 ('see', 19922),
 ('get', 17375),
 ('some', 15672),
 ('time', 14656),
 ('more', 13856),
 ('very', 13835),
 ('character', 13699),
 ('go', 13110),
 ('story', 12638),
 ('even', 12580),
 ('bad', 12324),
 ('only', 11780),
 ('really', 11534),
 ('watch', 10704),
 ('no', 10575),
 ('show', 10300),
 ('scene', 10215),
 ('look', 9564),
 ('much', 9550),
 ('say', 9496),
 ('people', 9210),
 ('great', 9201),
 ('also', 9082),
 ('know', 9069),
 ('other', 9044),
 ('end', 9040),
 ('most', 8727),
 ('take', 8528),
 ('give', 8523),
 ('play', 8392),
 ('way', 8285),
 ('act', 8223),
 ('come', 7980),
 ('well', 7919),
 ('find', 7907),
 ('first', 7896),
 ('think', 7666),
 ('love', 7652),
 ('too', 7224),
 ('seem', 7083),
 ('life', 7065),
 ('want', 6542),
 ('plot', 6493),
 ('actor', 6453),
 ('many', 6272),
 ('try', 6172),
 ('never', 6105),
 ('year',

In [209]:
most_common = counter.most_common(1000)
s = most_common
most_common = {}
most_common_reversed = {}
count = 0
for it in s:
    most_common[it[0]] = count
    most_common_reversed[count] = it[0]
    count+=1

In [210]:
for word in dictionary:
    if dictionary[word]>0 and dictionary[word]<len(reversed_dict):
        if reversed_dict[dictionary[word]] in most_common:
            dictionary[word] = most_common[reversed_dict[dictionary[word]]]
        else:
            dictionary[word] = -1
    else:
        dictionary[word] = -1

In [211]:
file_path = "./aclImdb/train/pos/5_10.txt"
content = open(file_path,encoding="utf8").read()
words = nltk.tokenize.word_tokenize(content)
preprocessed_sentence = "";
l = []
for word in words:
    if dictionary[word]>=0:
        preprocessed_sentence+=most_common_reversed[dictionary[word]]+" "
        l.append(dictionary[word])

In [212]:
preprocessed_sentence

'come middle film so no idea credit even title look <sep> see receive <end_sent> positive side regard film really catch attention watched beautiful score write <pn> <pn> style <end_sent> surprise great discover score write none other <pn> <pn> <end_sent> <pn> write score such <pn> <pn> usually name such <pn> <pn> <end_sent> opinion <pn> write movie anything ever heard <sep> beauty <sep> fully keep lovely plot movie <end_sent> another recent score <sep> <pn> <sep> show still more <end_sent> <pn> <pn> <sep> movie <pn> <pn> <sep> <pn> <pn> young american <pn> <pn> <sep> <pn> <pn> <pn> <sep> <end_sent> tell story <sep> story ca not told often enough <end_sent> excellent genre <end_sent> '

In [213]:
content

"I came in in the middle of this film so I had no idea about any credits or even its title till I looked it up here, where I see that it has received a mixed reception by your commentators. I'm on the positive side regarding this film but one thing really caught my attention as I watched: the beautiful and sensitive score written in a Coplandesque Americana style. My surprise was great when I discovered the score to have been written by none other than John Williams himself. True he has written sensitive and poignant scores such as Schindler's List but one usually associates his name with such bombasticities as Star Wars. But in my opinion what Williams has written for this movie surpasses anything I've ever heard of his for tenderness, sensitivity and beauty, fully in keeping with the tender and lovely plot of the movie. And another recent score of his, for Catch Me if You Can, shows still more wit and sophistication. As to Stanley and Iris, I like education movies like How Green was 